# Breve Introdução

**USE PYSPARK OU SQL PARA RESOLVER AS QUESTÕES ABAIXO**


**NENHUM DADO DISPONIBILIZADO É REAL**

Uma das áreas de negócio da XP é o B2B, onde empresas denominadas de escritórios de investimentos, fazem uma parceria com a XP para que seus clientes invistam através de nossa plataforma. Esses escritórios atendem nossos clientes com os assessores autônomos de invesmento, que ajudam as pessoas a investirem melhor. Depois de uma reunião com um dos gerentes da nossa área B2B, ficou claro que a carteira de muitos clientes estava desalinhada com as carteiras recomendadas para seu perfil e caso conseguíssemos reduzir esse desalinhamento, iriamos ajudar os clientes tornando suas carteiras mais seguras e rentáveis ao mesmo tempo que aumentaríamos nossa receita. Criamos então um projeto separado em 4 etapas para conseguirmos aumentar a aderência das carteiras de invesmento dos clientes dentro de seu suitability.


**Entendendo melhor a nosso banco de dados**

As tabelas disponibilizadas para esse desafio como você pode perceber na query acima possuem dois prefixos diferentes, sendo eles; FAT_ e DIM_

* **FAT_**: São as tabelas fato, o que significa que são tabelas que guardam os fatos históricos de algum acontecimento. Então de forma periódica algumas informações podem se repetir
  
  Exemplo: Se estivéssemos falando de uma tabela fato de vendas por cliente, teríamos uma tabela onde cada linha seria o total de vendas de um cliente naquele dia, então o mesmo cliente poderá se repetir na tabela várias vezes, porque as vendas podem acontecer em vários dias diferentes.
  
* **DIM_**: São as tabelas dimensão, elas se relacionam com as tabelas fato e guardam informações sobre algo, geralmente são tabelas com menos linhas e mais colunas que as fato.

  Exemplo: Pegando o exemplo de vendas da fato explicada acima. Na fato a conta do cliente vai se repetir várias vezes por conta dos dias, então é mantido na tabela fato apenas o código do cliente e na tabela dimensão vão ter todas as informações sobre os clientes, por exemplo o nome, idade, profissão e etc... dessa forma essas informações de texto não vão se repetir na tabela fato tornando a tabela muito mais rápida e menor.
  
**O que são os COD_DIM?**

Você vai perceber que em nossas tabelas existem colunas com o nome COD_DIM_, essas colunas são códigos que são usados para se relacionar dimensões e fatos, então por exemplo a coluna COD_DIM_TEMPO da tabela DIM_TEMPO, também existe na tabela fato FAT_NET_MENSAL e é usada para levar informações sobre o tempo para a tabela fato, que por sua vez fica mais rápida por não ter dados de texto.

# Preparação (Desafio 0)

Após ter criado o cluster e carregado os arquivos CSV para o Databricks chegou a hora de transformá-los em tabelas.
Use a célula abaixo ou (crie células, se necessário) para transformar os arquivos fornecidos em Tabelas do Databricks.




Quando uma tabela é criada o resultado da query fica como "Query returned no results", não se preocupe, isso significa que a tabela foi criada com sucesso, você pode consulta-la após isso fazendo um select, como mostrado na célula a seguir

In [0]:
%python

def read_csv(file_name:str):
    '''
    Read the csv file loaded into FileStore location and outputs an spark dataframe
    '''
    # File location and type
    file_location = "/FileStore/tables/{}.csv".format(file_name)
    file_type = "csv"
    
    # CSV options
    infer_schema = "true"
    first_row_is_header = "true"
    delimiter = ";"
    
    # The applied options are for CSV files. For other file types, these will be ignored.
    return spark.read.format(file_type) \
    .option("inferSchema", infer_schema) \
    .option("header", first_row_is_header) \
    .option("sep", delimiter) \
    .load(file_location)

In [0]:
%python

lista_nomes_tabela = [
    'fat_recomendacao',
    'fat_net_mensal',
    'dim_produto',
    'dim_conta_suitability',
    'dim_canal_assessor'
]

for nome_tabela in lista_nomes_tabela:
    df = read_csv(file_name = nome_tabela)
    (
        df.write
          .mode('overwrite')
          .option("overwriteSchema", "true")
          .saveAsTable(nome_tabela)
    )

<br>
Para finalizar nossa preparação, essa etapa consiste em criar uma tabela que lista o suitability e a custódia mensal dos clientes do B2B para cada estratégia diferente de produto.
Portanto, é importante identificar qual (ou quais) tabela(s) fato ou dimensão você irá usar.

**Notas:**
- Suitability é o perfil do investidor, pode ser conservador, moderado ou agressivo e representa o quanto de risco o cliente está disposto a assumir pelo retorno
- Custódia é o tanto de dinheiro dos clientes que está investido dentro da plataforma XP. Esse dinheiro não é da XP
- Estratégia de produto é um tipo de classificação de produto que assume as seguintes categorias: Prefixado, Pós-Fixado, Inflação, Multimercado, Renda Variável e Internacional
- Se o assessor de algum cliente é do B2B, então esse cliente também é considerado desse canal


*Lembre-se de antes de usar esse notebook ativar o cluster, para isso uma vez que ele já tenha sido criado, basta clicar no botão "Detached" encontrado no canto esquerdo superior abaixo do título do notebook. Para mais informações olhe o tutorial em word*

Agora, com as tabelas tabelas fato e dimensão criadas crie a *fat_custodia_conta_mensal*:

##### Atenção: Nomeie essa tabela com o seguinte nome: fat_custodia_conta_mensal

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.window import Window

In [0]:
# OUTPUT -> fat_custodia_conta_mensal
# OUTPUT COLUMNS:
#     COD_DIM_TEMPO
#     COD_DIM_CONTA
#     CANAL
#     SUITABILITY
#     CLASSE_N1
#     VAL_POSICAO


# AUX FUNC
def show_tb(df):
    df.show(10)

# Dependencies
df_canal_assessor = spark.read.table('dim_canal_assessor').where(F.col('CANAL') == F.lit('B2B'))
df_conta_suitability = spark.read.table('dim_conta_suitability')
df_produto = spark.read.table('dim_produto')
df_net_mensal = spark.read.table('fat_net_mensal')


df_custodia_conta_mensal = (
    df_net_mensal
    .join(
        df_canal_assessor,
        'COD_DIM_CANAL_ASSESSOR',
        'inner'
    )
    .join(
        df_conta_suitability,
        'COD_DIM_CONTA',
        'left'
    )
    .join(
        df_produto,
        'COD_DIM_PRODUTO',
        'left'
    )
    .select(
        F.col('COD_DIM_TEMPO'),
        F.col('COD_DIM_CONTA'),
        F.col('SUITABILITY'),
        F.col('CANAL'),
        F.col('CLASSE_N1'),
        F.col('VAL_POSICAO')
    )
)


# Write Table
nome_tabela = 'fat_custodia_conta_mensal'
(
    df_custodia_conta_mensal
    .write
    .mode('overwrite')
    .option("overwriteSchema", "true")
    .saveAsTable(nome_tabela)
)

# Desafio 1

Usando a tabela gerada na primeira etapa com o nome de *fat_custodia_conta_mensal*, faça um agrupamento de tal forma que 1) exiba a custódia mais recente do cliente independente da estratégia e 2) em seguida crie uma coluna que exiba a alocação em cada estratégia de produto para cada cliente.

Alocação: % de custódia que o produto tem na carteira

Layout de resultado esperado:

| COD_DIM_CONTA | SUITABILITY | ESTRATEGIA     | CUSTODIA  | ALOCACAO            |
|---------------|-------------|----------------|-----------|---------------------|
| 13            | Moderado    | Internacional  | 8478.79   | 0.046 |
| 13            | Moderado    | Pós-Fixado     | 116706.06 | 0.641  |
| 13            | Moderado    | Renda Variável | 56791.49  | 0.312   |
| 14            | Moderado    | Internacional  | 848.92   | 0.046 |
| 14            | Moderado    | Pós-Fixado     | 116.61 | 0.641  |
| 14            | Moderado    | Renda Variável | 791.44  | 0.312   |

In [0]:
# CASO 1
#### CASO EXTRA: CASOS QUE NAO TEM A ESTRATEGIA NO MES, DEVO PREENCHER COM 0 OU NÃO É RELEVANTE


wind = Window.partitionBy("COD_DIM_CONTA", "CLASSE_N1")

df_custodia_mensal_por_estrat = (
    df_custodia_conta_mensal
    .withColumn(
        'last_n1_dt',
        F.max('COD_DIM_TEMPO').over(wind)
    ).where(
        F.col('last_n1_dt') == F.col('COD_DIM_TEMPO')
    )
    .groupBy(
        F.col('COD_DIM_CONTA'),
        F.col('CLASSE_N1'),
        F.col('SUITABILITY')
    )
    .agg(
        F.sum('val_posicao').alias('CUSTODIA')
    )
    .withColumn(
        'total_cliente',
        F.sum('CUSTODIA').over(Window.partitionBy('COD_DIM_CONTA'))
    )
    .withColumn(
        'ALOCACAO',
        F.col('CUSTODIA') / F.col('total_cliente')
    )
    .drop('total_cliente')
    .distinct()
)





# ###### CASO 2

# df_last_custody_strategy = (
#     df_custodia_conta_mensal
#     .groupBy(
#         F.col('COD_DIM_CONTA'),
#         F.col('CLASSE_N1'),
#         F.col('SUITABILITY')
#     )
#     .agg(
#         F.max(F.col('COD_DIM_TEMPO')).alias('COD_DIM_TEMPO')
#     )
# )

# df_last_custody_distribution = (
#     df_last_custody_strategy
#     .join(
#         df_custodia_conta_mensal,
#         [
#             'COD_DIM_TEMPO',
#             'COD_DIM_CONTA',
#             'CLASSE_N1',
#             'SUITABILITY'
#         ],
#         'inner'
#     )
#     .groupBy(
#         F.col('COD_DIM_CONTA'),
#         F.col('CLASSE_N1'),
#         F.col('SUITABILITY')
#     )
#     .agg(F.sum(F.col('val_posicao')).alias('CUSTODIA'))
#     .withColumn(
#         'Total_cliente',
#         F.sum('CUSTODIA').over(Window.partitionBy('cod_dim_conta'))
#     )
#     .withColumn(
#         'ALOCACAO',
#         F.col('CUSTODIA') / F.col('Total_cliente')
#     )
#     .drop('Total_cliente')
# )





# Desafio 2

Calcule a diferença de alocação em cada estratégia de produto baseado na alocação recomendada pela tabela *FAT_RECOMENDACAO*. Essa nova coluna deve mostrar a diferença em pontos percentuais (valor absoluto) entre a carteira do cliente e a carteira recomendada. Após isso filtre os 5 clientes com maior diferença de alocação.

**Notas:**
- Para filtrar os 5 clientes com maior diferença de alocação é preciso descobrir a diferença de alocação total por cliente (desconsiderando a quebra de estratégia).
- Adote como premissa que vamos usar a recomendação mais recente da fat_recomendacao, você pode fazer isso filtrando a data máxima

Nessa etapa você precisará usar o resultado do exercício anterior. 

Além de usar o resultado do exercício anterior, use a tabela fat_recomendacao disponibilizada para chegar ao resultado esperado, lembrando que a recomendação é feita para cada classe de ativo de acordo com o suitability e isso pode mudar todos os meses


In [0]:
df_recomendacao = spark.read.table('fat_recomendacao')

# Ver o que fica mais performatico depois, se é um group by com join ou window function

df_recomendacao = (
    df_recomendacao
    .withColumn(
        'dt_max',
        F.max(F.col('cod_dim_tempo')).over(Window.partitionBy('SUITABILITY'))
    )
    .where(F.col('COD_DIM_TEMPO') == F.col('dt_max'))
    .drop('dt_max')
)


df_calc_desvio = (
    df_custodia_mensal_por_estrat
    .join(
        df_recomendacao,
        ['SUITABILITY', 'CLASSE_N1'],
        'left'
    )
    .withColumn(
        'DESVIO_ALOCADO',
        F.abs(F.col('ALOCACAO') - F.col('ALOCACAO_RECOMENDADA'))
    )
    .drop('COD_DIM_TEMPO')
)

df_top_5 = (
    df_calc_desvio
    .withColumn(
        'TOTAL_DESVIO_CLIENTE',
        F.sum(F.col('DESVIO_ALOCADO')).over(Window.partitionBy('COD_DIM_CONTA'))
    )
    .withColumn(
        'rn',
        F.dense_rank().over(Window.orderBy(F.col('TOTAL_DESVIO_CLIENTE').desc()))
    )
    .where(F.col('rn') <= F.lit(5))
)

df_top_5.show(20)

+-----------+--------------+-------------+------------------+-------------------+--------------------+-------------------+--------------------+---+
|SUITABILITY|     CLASSE_N1|COD_DIM_CONTA|          CUSTODIA|           ALOCACAO|ALOCACAO_RECOMENDADA|     DESVIO_ALOCADO|TOTAL_DESVIO_CLIENTE| rn|
+-----------+--------------+-------------+------------------+-------------------+--------------------+-------------------+--------------------+---+
|   Moderado| Internacional|          128|          126687.8|0.35113728298173585|                 0.1| 0.2511372829817359|  1.2425486218834916|  1|
|   Moderado|    Pós-Fixado|          128|          64482.94|0.17872568905825417|                 0.8| 0.6212743109417459|  1.2425486218834916|  1|
|   Moderado|Renda Variável|          128|169622.05000000002|0.47013702796000995|                 0.1| 0.3701370279600099|  1.2425486218834916|  1|
|Conservador| Internacional|          135|          42395.36|0.13800940402644835|                 0.1|0.03800940

Layout de resultado esperado:

| COD_DIM_CONTA | SUITABILITY | ESTRATEGIA     | CUSTODIA | ALOCACAO            | ALOCACAO_RECOMENDADA | DESVIO_ALOCACAO     |
|---------------|-------------|----------------|----------|---------------------|----------------------|---------------------|
| 40            | Conservador | Internacional  | 51676.24 | 0.207 | 0.1                  | 0.107 |
| 40            | Conservador | Pós-Fixado     | 81623.5  | 0.328 | 0.9                  | 0.571  |
| 40            | Conservador | Renda Variável | 115475.6 | 0.464  | 0                   | 0.464  |

In [0]:
%sql
use catalog `hive_metastore`; select * from `default`.`fat_recomendacao` limit 100;

COD_DIM_TEMPO,SUITABILITY,CLASSE_N1,ALOCACAO_RECOMENDADA
20210131,Conservador,Renda Variável,0.0
20210131,Conservador,Pós-Fixado,0.96
20210131,Conservador,Internacional,0.04
20210228,Conservador,Renda Variável,0.0
20210228,Conservador,Pós-Fixado,0.95
20210228,Conservador,Internacional,0.05
20210331,Conservador,Renda Variável,0.0
20210331,Conservador,Pós-Fixado,0.85
20210331,Conservador,Internacional,0.15
20210430,Conservador,Renda Variável,0.0


# Desafio 3

Bem agora que você construíu os dados de alocação recomendada e sabe melhor como está a carteira dos clientes, quais seriam os próximos passos? Sinta-se a vontade para usar todas as tabelas disponibilizadas até então.

Além disso o time de produtos te disponibilizou uma tabela com o ROA médio para cada estratégia de produto. Essa tabela pode ser acessada com o nome de *TB_ROA_MEDIO*

**Notas:**
- ROA = Revenue Over Assets = Receita / Custódia
- Receita = Dinheiro que a XP ganha com taxas, ou operações do cliente
- Custódia = Dinheiro dos clientes que está investido dentro da plataforma XP

**Sugestões de evolução:**

Sinta-se à vontade para fazer qualquer uma das sugestões ou ainda qualquer outra ideia que você tiver, use a criatividade e mostre seus conhecimentos técnicos, lembre-se que aqui no databricks você pode usar não só o SQL mas também o Python ou R caso você tenha conhecimento sob essas linguagens. Se achar válido explore seus conhecimentos em estatística, Pandas, Sklearn ou outros.

Algumas sugestões:
- Calcular a receita potencial gerada com a alocação recomendada dos clientes, ou seja, hoje os clientes possuem um desvio para a alocação recomendada, se esses clientes vendessem seus ativos e comprassem outros quanto de receita ele geraria para a XP? Você pode usar a tabela de ROA médio por estratégia de produto para fazer esse cálculo
- Analisar quais eram os clientes com maior custódia total por suitability
- Analisar os produtos com maior alocação dos clientes dentro do canal B2C
- Analisar o histórico de distribuição de produtos do cliente com os maiores desvios atualmente
- Exibir alguns gráficos usando o databricks ou o python
- Fazer uma projeção de custódia total do canal B2B e B2C